AI Technical Explainer



In [ ]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr



In [ ]:

MODEL = "gpt-4o-mini"


In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv("********")
if api_key:
    print(f"api key exists and begins with {api_key[:8]}")
else:
    print("api key is not set")

openai = OpenAI()
EXPLAIN_MODEL = MODEL



api key exists and begins with sk-proj-


In [ ]:
# ---- System prompt ----
explain_system_prompt = (
    "You are a concise technical explainer. Given a technical question or code snippet, "
    "produce a clear, accurate explanation with the following structure:\n"
    "1) What it is/does\n"
    "2) How it works (step-by-step)\n"
    "3) Why it’s written that way (tradeoffs, idioms)\n"
    "4) Pitfalls & edge cases\n"
    "5) Tiny example(s) if helpful\n"
    "Keep it correct, brief, and formatted in Markdown."
)




In [ ]:
def get_explainer_user_prompt(question: str, context: str = "") -> str:
    q = (question or "").strip()
    ctx = (context or "").strip()
    user_prompt = "You are looking at a technical question.\n"
    user_prompt += "Explain it for a practitioner with clear headings.\n\n"
    user_prompt += "Question:\n" + q + "\n"
    if ctx:
        user_prompt += "\nAdditional context (if any):\n" + ctx + "\n"
    return user_prompt[:8000]


In [ ]:
def _coerce_messages_to_q_and_ctx(messages):
    """
    Accepts either:
      - list[{'role': ..., 'content': ...}, ...]  (gr.ChatInterface type='messages')
      - str                                     (gr.ChatInterface type='text' or older versions)
    Returns: (question_str, context_str)
    """
    # Case 1: list of dicts (expected for type='messages')
    if isinstance(messages, list):
        # Some builds may pass plain strings inside; be defensive
        cleaned = []
        for m in messages:
            if isinstance(m, dict):
                cleaned.append(m)
            elif isinstance(m, str):
                cleaned.append({"role": "user", "content": m})
            else:
                cleaned.append({"role": "user", "content": str(m)})

        user_msgs = [m for m in cleaned if m.get("role") == "user" and isinstance(m.get("content"), str)]
        question = user_msgs[-1]["content"] if user_msgs else ""

        context_parts = []
        # Everything before last user message as context
        for m in cleaned[:-1]:
            c = m.get("content", "")
            if isinstance(c, str) and c.strip():
                context_parts.append(c)
        context = "\n\n".join(context_parts)
        return question, context

    # Case 2: plain string (type='text' or some frontends)
    if isinstance(messages, str):
        return messages, ""  # question is the whole string, no prior context

    # Fallback: unknown shape
    return str(messages), ""


In [ ]:

def stream_explanation(messages, state=None, *args, **kwargs):
    """
    Streaming function compatible with multiple Gradio versions.
    Accepts either list[dict] or str and yields partial markdown.
    """
    question, context = _coerce_messages_to_q_and_ctx(messages)

    # 👋 Custom greeting if user just says "hi" or similar
    if question.strip().lower() in {"hi", "hello", "hey"}:
        yield "I am a technical explainer — ask any technical question to get your answer."
        return

    stream = openai.chat.completions.create(
        model=EXPLAIN_MODEL,
        messages=[
            {"role": "system", "content": explain_system_prompt},
            {"role": "user", "content": get_explainer_user_prompt(question, context)},
        ],
        stream=True,
        temperature=0.2,
    )

    partial = ""
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""
        if not delta:
            continue
        partial += delta
        # clean occasional fences
        cleaned = partial.replace("```markdown", "").replace("```", "")
        yield cleaned



In [ ]:
# -------- Gradio UI (streaming) --------
demo = gr.ChatInterface(
    fn=stream_explanation,          # generator for streaming
    type="messages",                # history as list of dicts (we also handle str fallback)
    title="Your AI Technical Explainer",
    description="Ask me any technical question.",
    examples=[
        [{"role": "user", "content": "Explain Python's list comprehensions vs map/filter."}],
        [{"role": "user", "content": "What does this regex do? ^(?!.*password).*$"}],
        [{"role": "user", "content": "Give me the gist of Python asyncio."}],
    ],
    cache_examples=False,
    submit_btn="Explain ▶",
)

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
